### 1. Import library

In [1]:
import pandas as pd
import numpy as np
from datetime import date

---
### 2. Load dataset

In [2]:
df_fbl5n = pd.read_excel('CAOT Test Case _ Final.xlsx',sheet_name='Input Data - FBL5N')

df_cust = pd.read_excel('CAOT Test Case _ Final.xlsx',sheet_name='Input Data - Cust Hierarchy')

df_holiday = pd.read_excel('CAOT Test Case _ Final.xlsx',sheet_name='Input - Holiday')

df_reportdate = pd.read_excel('CAOT Test Case _ Final.xlsx',sheet_name='Input - Report Date',header=None)

df = df_fbl5n.copy()


---
### 3. Create [[Customer Name, Trade Format Level 2, Channel, Sales Workforce Lv2, Sales Workforce Lv3, Banner Name]]

In [3]:
# create support
dict_values = tuple(zip(
    df_cust['Customer Name'],
    df_cust['TFL2'],
    df_cust['Customer Channel'],
    df_cust['Customer Channel'],
    df_cust['SWL2'],
    df_cust['SWL3'],
    df_cust['Banner'],
))

dict_keys_values = dict(zip(df_cust['Customer'],dict_values))

# create function
def name(account):
    for key, value in dict_keys_values.items():
        if account == key:
            return value[0]

def tfl2(account):
    for key, value in dict_keys_values.items():
        if account == key:
            return value[1]

def channel(account):
    for key, value in dict_keys_values.items():
        if account == key:
            return value[2]

def swl2(account):
    for key, value in dict_keys_values.items():
        if account == key:
            return value[3]

def swl3(account):
    for key, value in dict_keys_values.items():
        if account == key:
            return value[4]

def banner(account):
    for key, value in dict_keys_values.items():
        if account == key:
            return value[5]

In [4]:
# apply function
df.insert(2,'Banner Name',df['Account'].apply(banner))
df.insert(2,'Sales Workforce Lv3',df['Account'].apply(swl3))
df.insert(2,'Sales Workforce Lv2',df['Account'].apply(swl2))
df.insert(2,'Channel',df['Account'].apply(channel))
df.insert(2,'Trade Format Level 2',df['Account'].apply(tfl2))
df.insert(2,'Customer Name',df['Account'].apply(name))

df.head(3)

,Company Code,Account,Customer Name,Trade Format Level 2,Channel,Sales Workforce Lv2,Sales Workforce Lv3,Banner Name,Billing Document,Document Number,...,Sales Document,Vendor,Customer,Reference,Text,Clearing Document,Clearing date,Arrears after net due date,Reference Key 1,Reference Key 2
0,2408,15061017,PE. SXGKCA QLCQAWQ EAWJQYQ YC KQIYX,GZYCAI EAQYC,MT,MT,CYG GWIWGQAKCE,KQG QLCQGQAE,NaN,3400015095,...,NaN,NaN,15061017,683040,683040,3.400015e+09,2023-02-01,0,NaN,NaN
1,2408,15061063,PE. EXLXS GQJX,YWSEAWKXEWVC EAQYC,UFS,UFS,AKG CCIEAQL,QKG CQSECAI JQKQAEQ,NaN,3400015047,...,NaN,NaN,15061063,682951,682951,3.400015e+09,2023-02-01,0,NaN,NaN
2,2408,15061820,CLQIYYS GQAE,GZYCAI EAQYC,MT,MT,ASG ZXECA WSLQIY,KXH KQLW,NaN,3400014851,...,NaN,NaN,15061820,NaN,NaN,3.400015e+09,2023-02-01,0,NaN,NaN


---
### 4. Create [[Exclude, Days, Ageing, Clearing Status]]

In [95]:
# create support
reportdate = df_reportdate[0][0]
dayoff = [1,1,1,1,1,0,0]
weekmask = 'Mon Tue Wed Thu Fri'

# create function
def rule(x):
    if str(x['Reference Key 1']).upper().find('CITDUMMY') != -1:
        return x['Reference Key 1']
    elif float(x['Amount in local currency']) >= 0:
        return 'Positive Value'
    elif str(x['Local Currency']) != 'IDR' or str(x['Document currency']) != 'IDR':
        return 'Not IDR'
    elif str(x['Document Number']).startswith('58') == True:
        return 'Reversal Document'
    elif str(x['Reference']).upper().find('BNPPAY') != -1 or str(x['Text']).upper().find('BNPPAY') != -1:
        return 'BNPPAY'
    else :
        return np.NaN

def to_date(x):
    return x.date()

def days(x):

    holidaylist = df_holiday['Holiday'].apply(to_date).to_list()

    if pd.notna(x['Clearing Document'])==True:

        busday = np.busday_count(
            x['Document Date'].date(),x['Clearing date'].date(),weekmask,holidays=holidaylist
        )

        if busday <= 0:
            return 0
        else :
            return busday

    else :

        busday = np.busday_count(
            x['Document Date'].date(),reportdate.date(),weekmask,holidays=holidaylist
        )

        if busday <= 0:
            return 0
        else :
            return busday

def ageing(x):
    if x['Days'] <= -1:
        return '>2'

    elif x['Days'] < 2:
        return '<2'

    elif x['Days'] == 2:
        return '=2'

    elif x['Days'] >= 3:
        return '>2'

def clearing(x):

    if pd.notna(x['Clearing Document'])==True:
        return 'Cleared'

    else :
        return 'Open'

In [96]:
# apply function
df['Exclude'] = df.T.apply(rule)
df['Days'] = df.T.apply(days)
df['Ageing'] = df.T.apply(ageing)
df['Clearing Status'] = df.T.apply(clearing)

df.head(3)

,Company Code,Account,Customer Name,Trade Format Level 2,Channel,Sales Workforce Lv2,Sales Workforce Lv3,Banner Name,Billing Document,Document Number,...,Text,Clearing Document,Clearing date,Arrears after net due date,Reference Key 1,Reference Key 2,Exclude,Days,Ageing,Clearing Status
0,2408,15061017,PE. SXGKCA QLCQAWQ EAWJQYQ YC KQIYX,GZYCAI EAQYC,MT,MT,CYG GWIWGQAKCE,KQG QLCQGQAE,NaN,3400015095,...,683040,3.400015e+09,2023-02-01,0,NaN,NaN,Not IDR,0,<2,Cleared
1,2408,15061063,PE. EXLXS GQJX,YWSEAWKXEWVC EAQYC,UFS,UFS,AKG CCIEAQL,QKG CQSECAI JQKQAEQ,NaN,3400015047,...,682951,3.400015e+09,2023-02-01,0,NaN,NaN,Positive Value,0,<2,Cleared
2,2408,15061820,CLQIYYS GQAE,GZYCAI EAQYC,MT,MT,ASG ZXECA WSLQIY,KXH KQLW,NaN,3400014851,...,NaN,3.400015e+09,2023-02-01,0,NaN,NaN,NaN,0,<2,Cleared


In [97]:
df['Days'].value_counts()

0    2123
2     249
1     230
3     211
Name: Days, dtype: int64

In [67]:
holidaylist = df_holiday['Holiday'].apply(to_date).to_list()
np.busday_count(
            df['Document Date'][0].date(),df['Clearing date'][0].date(),weekmask='Mon Tue Wed Thu Fri',holidays=holidaylist
        )

0

---
### 5. Save as CSV and XLSX

In [7]:
pd.DataFrame.to_csv(df,'unvr CAOT - output.csv',index=False)
pd.DataFrame.to_excel(df,'unvr CAOT - output.xlsx',index=False)